In [4]:
import cv2
import os
import tkinter as tk
from tkinter import filedialog, messagebox
from tkinter import ttk
import threading

stop_event = threading.Event()

def browse_videos():
    files = filedialog.askopenfilenames(filetypes=[("Video files", "*.mp4 *.avi *.mov")])
    if not files:
        return
    video_paths.set(";".join(files))
    update_video_info(files)

def browse_output_dir():
    output_dir.set(filedialog.askdirectory())

def update_video_info(files):
    for i in tree.get_children():
        tree.delete(i)
    
    total_target_frames = 0
    for video_path in files:
        video = cv2.VideoCapture(video_path)
        original_fps = video.get(cv2.CAP_PROP_FPS)
        total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
        video.release()
        
        target_fps_val = float(target_fps.get())
        if target_fps_val == 0:
            continue
        frame_interval = int(original_fps / target_fps_val)
        target_frame_count = total_frames // frame_interval
        total_target_frames += target_frame_count
        
        tree.insert("", "end", values=(os.path.basename(video_path), original_fps, total_frames, target_frame_count))
    
    total_export_frame_label.config(text=f"Total Export Frames: {total_target_frames}")

def extract_frames():
    video_paths_val = video_paths.get().split(";")
    output_dir_val = output_dir.get()
    target_fps_val = float(target_fps.get())
    
    if not video_paths_val or not output_dir_val or not target_fps_val:
        messagebox.showerror("Error", "All fields are required")
        return
    
    if not os.path.exists(output_dir_val):
        os.makedirs(output_dir_val)
    
    total_frames_to_extract = 0
    for video_path in video_paths_val:
        video = cv2.VideoCapture(video_path)
        original_fps = video.get(cv2.CAP_PROP_FPS)
        total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_interval = int(original_fps / target_fps_val)
        total_frames_to_extract += total_frames // frame_interval
        video.release()
    
    progress_bar["maximum"] = total_frames_to_extract
    progress_bar["value"] = 0
    status_label.config(text="Extracting...")
    stop_event.clear()

    def process_video(video_path):
        video = cv2.VideoCapture(video_path)
        original_fps = video.get(cv2.CAP_PROP_FPS)
        total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_interval = int(original_fps / target_fps_val)
        target_frame_count = total_frames // frame_interval
        
        frame_count = 0
        saved_frame_count = 0
        success, frame = video.read()
        
        video_name = os.path.splitext(os.path.basename(video_path))[0]
        video_output_dir = os.path.join(output_dir_val, video_name)
        if not os.path.exists(video_output_dir):
            os.makedirs(video_output_dir)
        
        while success and not stop_event.is_set():
            if frame_count % frame_interval == 0:
                cv2.imwrite(os.path.join(video_output_dir, f"frame_{saved_frame_count}.jpg"), frame)
                saved_frame_count += 1
                progress_bar["value"] += 1
                status_label.config(text=f"Extracting... {progress_bar['value']}/{total_frames_to_extract} frames")
                root.update_idletasks()
            
            success, frame = video.read()
            frame_count += 1
        
        video.release()
        return target_frame_count

    def run_extraction():
        total_target_frames = 0
        for video_path in video_paths_val:
            if stop_event.is_set():
                break
            target_frame_count = process_video(video_path)
            total_target_frames += target_frame_count
        
        status_label.config(text="Extraction Completed" if not stop_event.is_set() else "Extraction Stopped")
        if not stop_event.is_set():
            messagebox.showinfo("Success", "Frame extraction completed.")
    
    threading.Thread(target=run_extraction).start()

def stop_extraction():
    stop_event.set()

def on_fps_change(event):
    files = video_paths.get().split(";")
    if files:
        update_video_info(files)

# Create the main window
root = tk.Tk()
root.title("Frame Extractor")

# Define StringVars for entry fields
video_paths = tk.StringVar()
output_dir = tk.StringVar()
target_fps = tk.StringVar(value="25")

# Create a PanedWindow
paned_window = tk.PanedWindow(root, orient=tk.HORIZONTAL)
paned_window.pack(fill=tk.BOTH, expand=1)

# Create the left frame for controls
left_frame = tk.Frame(paned_window)
paned_window.add(left_frame)

# Create the right frame for video info
right_frame = tk.Frame(paned_window)
paned_window.add(right_frame)

# Create and place widgets in the left frame
tk.Label(left_frame, text="Video Files:").grid(row=0, column=0, padx=10, pady=5, sticky="e")
tk.Entry(left_frame, textvariable=video_paths, width=50).grid(row=0, column=1, padx=10, pady=5)
tk.Button(left_frame, text="Browse", command=browse_videos).grid(row=0, column=2, padx=10, pady=5)

tk.Label(left_frame, text="Output Directory:").grid(row=1, column=0, padx=10, pady=5, sticky="e")
tk.Entry(left_frame, textvariable=output_dir, width=50).grid(row=1, column=1, padx=10, pady=5)
tk.Button(left_frame, text="Browse", command=browse_output_dir).grid(row=1, column=2, padx=10, pady=5)

tk.Label(left_frame, text="Target FPS:").grid(row=2, column=0, padx=10, pady=5, sticky="e")
fps_options = ["15", "24", "25", "30", "60"]
fps_dropdown = ttk.Combobox(left_frame, textvariable=target_fps, values=fps_options, state="readonly")
fps_dropdown.grid(row=2, column=1, padx=10, pady=5)
fps_dropdown.current(2)  # Set default value to 25
fps_dropdown.bind("<<ComboboxSelected>>", on_fps_change)

tk.Button(left_frame, text="Extract Frames", command=extract_frames).grid(row=3, column=0, columnspan=2, pady=20)
tk.Button(left_frame, text="Stop", command=stop_extraction).grid(row=3, column=2, pady=20)

# Progress bar
progress_bar = ttk.Progressbar(left_frame, orient="horizontal", length=400, mode="determinate")
progress_bar.grid(row=4, column=0, columnspan=3, pady=10)

# Status label
status_label = tk.Label(left_frame, text="")
status_label.grid(row=5, column=0, columnspan=3, pady=5)

# Create and place widgets in the right frame
columns = ("#1", "#2", "#3", "#4")
tree = ttk.Treeview(right_frame, columns=columns, show="headings")
tree.heading("#1", text="Video File")
tree.heading("#2", text="Original FPS")
tree.heading("#3", text="Total Frames")
tree.heading("#4", text="Target Frames")
tree.pack(expand=True, fill="both")

total_export_frame_label = tk.Label(right_frame, text="Total Export Frames: N/A")
total_export_frame_label.pack(pady=5)

# Run the main loop
root.mainloop()